<a href="https://colab.research.google.com/github/EmperoR1127/ml_project/blob/master/Sample_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
# To support both python 2 and python 3
from __future__ import division, print_function, unicode_literals

# Common imports
import numpy as np
import os
from scipy.io import arff
import pandas as pd

# to make this notebook's output stable across runs
np.random.seed(42)
# Where to save the figures
PROJECT_ROOT_DIR = "/content/drive/My Drive/"

#load the dataset
path = PROJECT_ROOT_DIR + "Data/H-1B_Disclosure_FY18_RAW_Data.csv"
df = pd.read_csv(path, encoding='utf-8')
raw_df = df.copy()

#randomly select 20058 approved cases and 269 denied cases
df_approved = df[df['CASE_STATUS'] == "CERTIFIED"].sample(n = 2006, random_state = 42)
df_denied = df[df['CASE_STATUS'] != "CERTIFIED"].sample(n = 27, random_state = 42)
df = pd.concat([df_approved, df_denied], sort=False, axis=0).reset_index(drop=True)
df.to_csv(path_or_buf= PROJECT_ROOT_DIR + "ml_project_dataset/H-1B_Disclosure_RAW_Data.csv")

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (15,16) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [0]:
df.shape

(2033, 52)

In [0]:
raw_df = raw_df.drop(["CASE_NUMBER", "VISA_CLASS", 
                                        "EMPLOYER_NAME", "EMPLOYER_STATE","EMPLOYER_POSTAL_CODE", 
                                        "EMPLOYER_CITY", "EMPLOYER_BUSINESS_DBA", 
                                        "EMPLOYER_COUNTRY", "EMPLOYER_PROVINCE", "EMPLOYER_ADDRESS", 
                                        "EMPLOYER_PHONE", "EMPLOYER_PHONE_EXT", 
                                        "AGENT_ATTORNEY_NAME", "AGENT_ATTORNEY_CITY", "AGENT_ATTORNEY_STATE",
                                        "JOB_TITLE", "SOC_NAME",
                                        "PW_SOURCE", "PW_SOURCE_YEAR", "PW_SOURCE_OTHER", "WAGE_RATE_OF_PAY_FROM",
                                        "WAGE_RATE_OF_PAY_TO", "WAGE_UNIT_OF_PAY",
                                        "WORKSITE_CITY", "WORKSITE_COUNTY", "WORKSITE_POSTAL_CODE", 
                                        "ORIGINAL_CERT_DATE", "PUBLIC_DISCLOSURE_LOCATION"], axis=1)
#format EMPLOYMENT_START_DATE and EMPLOYMENT_END_DATE
raw_df['CASE_SUBMITTED'] = pd.to_datetime(raw_df['CASE_SUBMITTED'],infer_datetime_format=True,errors='coerce')
raw_df['DECISION_DATE'] = pd.to_datetime(raw_df['DECISION_DATE'],infer_datetime_format=True,errors='coerce')
raw_df['EMPLOYMENT_START_DATE'] = pd.to_datetime(raw_df['EMPLOYMENT_START_DATE'],infer_datetime_format=True,errors='coerce')
raw_df['EMPLOYMENT_END_DATE'] = pd.to_datetime(raw_df['EMPLOYMENT_END_DATE'],infer_datetime_format=True,errors='coerce')
#drop NaT rows because we can't "guess" the specific date
raw_df = raw_df[raw_df.CASE_SUBMITTED != 'NaT']
raw_df = raw_df[raw_df.DECISION_DATE != 'NaT']
raw_df = raw_df[raw_df.EMPLOYMENT_START_DATE != 'NaT']
raw_df = raw_df[raw_df.EMPLOYMENT_END_DATE != 'NaT']
#add one column as EMP_PERIOD, and drop EMPLOYMENT_START_DATE and EMPLOYMENT_END_DATE
raw_df['EMP_PERIOD'] = raw_df['EMPLOYMENT_END_DATE'] - raw_df['EMPLOYMENT_START_DATE']
raw_df['EMP_PERIOD'] = raw_df['EMP_PERIOD']/np.timedelta64(1,'Y')
#train_set = train_set[train_set.EMP_PERIOD != '-']
raw_df['EMP_PERIOD'] = raw_df['EMP_PERIOD'].astype(float)
#add one column as PROCESS_TIME, indicating processing time of visa application
raw_df['PROCESS_TIME'] = raw_df['DECISION_DATE'] - raw_df['CASE_SUBMITTED']
raw_df['PROCESS_TIME'] = raw_df['PROCESS_TIME'].map(lambda x: str(x)[:1])
raw_df['PROCESS_TIME'] = raw_df['PROCESS_TIME'].astype(float)
raw_df = raw_df.drop(["EMPLOYMENT_START_DATE", "EMPLOYMENT_END_DATE"], axis=1)
raw_df = raw_df.drop(["CASE_SUBMITTED", "DECISION_DATE"], axis=1)

#concatenate the first 2 digit of column SOC_CODE and NAIC_CODE
raw_df['SOC_CODE'] = raw_df['SOC_CODE'].map(lambda x: str(x)[:2])
raw_df['NAICS_CODE'] = raw_df['NAICS_CODE'].map(lambda x: str(x)[:2])
#remove impurity in the column
raw_df = raw_df[raw_df.PW_UNIT_OF_PAY != 'N']
raw_df = raw_df[raw_df.PREVAILING_WAGE != 'N']
#according to google, there are 2080 working hours per year
pw_unit_column = {"Year":1, "Hour":2080, "Month":12, "Week":52, "Bi-Weekly":26}
raw_df['PW_UNIT_OF_PAY'] = raw_df['PW_UNIT_OF_PAY'].replace(pw_unit_column)
#remove ',' in the column value
raw_df['PREVAILING_WAGE'] = raw_df['PREVAILING_WAGE'].astype('str')
raw_df['PREVAILING_WAGE'] = raw_df.PREVAILING_WAGE.str.replace(',','')
raw_df['PREVAILING_WAGE'] = raw_df['PREVAILING_WAGE'].astype('float')
#add one column as ANNUAL_SALARY
raw_df['ANNUAL_SALARY'] = raw_df['PREVAILING_WAGE'] * raw_df['PW_UNIT_OF_PAY']
raw_df = raw_df.drop(["PREVAILING_WAGE", "PW_UNIT_OF_PAY"], axis=1)


In [0]:
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder
from sklearn.compose import ColumnTransformer
raw_set = raw_df.drop(["CASE_STATUS"], axis=1)
raw_labels = raw_df[["CASE_STATUS"]].copy()
raw_set_num = raw_set.drop(["AGENT_REPRESENTING_EMPLOYER", "SOC_CODE", "NAICS_CODE",
                                "FULL_TIME_POSITION", "PW_WAGE_LEVEL", "H1B_DEPENDENT", "WILLFUL_VIOLATOR",
                                "SUPPORT_H1B", "LABOR_CON_AGREE", "WORKSITE_STATE"], axis=1)
raw_set_cat = raw_set.drop(["TOTAL_WORKERS","NEW_EMPLOYMENT","CONTINUED_EMPLOYMENT",
                                "CHANGE_PREVIOUS_EMPLOYMENT", "NEW_CONCURRENT_EMP", "CHANGE_EMPLOYER",
                                "AMENDED_PETITION", "EMP_PERIOD", "PROCESS_TIME",
                                "ANNUAL_SALARY"], axis=1)
#build the pipeline
num_pipeline = Pipeline([('imputer', SimpleImputer(strategy="median")),('std_scaler', StandardScaler()),])
cat_pipeline = Pipeline([('imputer', SimpleImputer(strategy="most_frequent")),('cat', OneHotEncoder()),])
full_pipeline = ColumnTransformer([("num", num_pipeline, list(raw_set_num)),("cat", cat_pipeline, list(raw_set_cat)),])

#prepare the data
raw_set = full_pipeline.fit_transform(raw_set)

#prepare the target
encoder = LabelEncoder()
raw_labels = encoder.fit_transform(raw_labels)

/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/label.py:235: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [0]:
raw_labels.shape

(574522,)

In [0]:
import joblib
joblib.dump(raw_set, PROJECT_ROOT_DIR + 'ml_project_dataset/' + 'raw_set' + '.gz', compress=('gzip', 3))
joblib.dump(raw_labels, PROJECT_ROOT_DIR + 'ml_project_dataset/' + 'raw_labels' + '.gz', compress=('gzip', 3))

['/content/drive/My Drive/ml_project_dataset/raw_labels.gz']

In [0]:
import joblib
raw_set = joblib.load(PROJECT_ROOT_DIR + 'ml_project_dataset/' + 'raw_set' + '.gz')

In [0]:
raw_set.shape

(574522, 139)